In [1]:
from google.colab import drive
import os, glob
import pathlib
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/scannnn/penalizing_top_performers.git

Cloning into 'penalizing_top_performers'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 166 (delta 102), reused 134 (delta 70), pack-reused 0
Receiving objects: 100% (166/166), 44.31 KiB | 2.01 MiB/s, done.
Resolving deltas: 100% (102/102), done.


In [3]:
%cd penalizing_top_performers

/content/penalizing_top_performers


In [4]:
!git checkout dev

Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


In [5]:
!pip install tensorboardX

     |████████████████████████████████| 124 kB 5.3 MB/s 


In [6]:
!pip install yacs

In [7]:
from util.logger import Logger
import utils
from configs import cfg
from torch.utils.data import DataLoader
import torch
from segmentation.trainer import Trainer
from train_fcn8 import *
from util.checkpoint import *
import copy

In [ ]:
source_img_path = "/content/gdrive/MyDrive/AI_PROJECT(BLG_527E)/dataset/source/images"
source_label_path ="/content/gdrive/MyDrive/AI_PROJECT(BLG_527E)/dataset/source/labels"
target_img_path = "/content/gdrive/MyDrive/AI_PROJECT(BLG_527E)/dataset/target/images"

if __name__ == '__main__':
    device = 'cuda'
    batch_size = 1
    n_classes = 19
    num_epochs = 100
    pretrained = True
    fixed_feature = False
    epoch = 0

    logger = Logger(model_name="fcn8_vgg16", data_name='gta5')

    src_train_imgs = utils.read_path(source_img_path,"png")
    src_train_labels = utils.read_path(source_label_path,"png")
    target_train_imgs = utils.read_path(target_img_path,"jpg")

    src_train_ds = utils.GTA5Dataset(cfg, src_train_imgs, src_train_labels)
    trgt_train_ds = utils.Dataset(cfg, target_train_imgs)



    src_train_loader = DataLoader(src_train_ds, batch_size=1, shuffle=True, drop_last=True)
    tgt_train_loader = DataLoader(trgt_train_ds, batch_size=1, shuffle=True, drop_last=True)

    ### Model
    model = fcn8_vgg16(n_classes)
    model.to(device)

    ###Load model
    ###please check the foloder: (.segmentation/test/runs/models)
    #logger.load_model(model, 'epoch_3')

    ### Optimizers
    if pretrained and fixed_feature: #fine tunning
        params_to_update = model.parameters()
        print("Params to learn:")
        params_to_update = []
        for name, param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t", name)
        optimizer = torch.optim.Adadelta(params_to_update)
    else:
        optimizer = torch.optim.Adadelta(model.parameters())

    ### Train
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    trainer = Trainer(model, optimizer, logger, num_epochs, src_train_loader, epoch=0)
    trainer.train()

In [ ]:
%cd /content/
!ls -l

In [ ]:
main(v2=True)

In [11]:
classifier_model = fcn8_vgg16(n_classes=19)

In [12]:
hdl_chkpoint = CheckpointHandler()
dir = '/content/gdrive/MyDrive/AI_PROJECT(BLG_527E)/fcn8vgg16_output/only_50/models/fcn8_vgg16/gta5/epoch_70'
hdl_chkpoint = hdl_chkpoint.load_checkpoint(dir)

classifier_model.load_state_dict(hdl_chkpoint.model_state_dict)
if hasattr(hdl_chkpoint, '__dict__'):
  for k in hdl_chkpoint.__dict__:
    if k == 'model_state_dict' or k == 'optimizer_state_dict':
      continue
    attr_copy = copy.deepcopy(getattr(hdl_chkpoint, k))
    setattr(classifier_model, k, attr_copy)

In [13]:
import cv2
import random
import numpy as np
import six

In [14]:
class_colors = [(random.randint(0, 255), random.randint(
    0, 255), random.randint(0, 255)) for _ in range(5000)]
colormap = np.zeros((256, 3), dtype=np.uint8)
colormap[0] = [128, 64, 128]
colormap[1] = [244, 35, 232]
colormap[2] = [70, 70, 70]
colormap[3] = [102, 102, 156]
colormap[4] = [190, 153, 153]
colormap[5] = [153, 153, 153]
colormap[6] = [250, 170, 30]
colormap[7] = [220, 220, 0]
colormap[8] = [107, 142, 35]
colormap[9] = [152, 251, 152]
colormap[10] = [70, 130, 180]
colormap[11] = [220, 20, 60]
colormap[12] = [255, 0, 0]
colormap[13] = [0, 0, 142]
colormap[14] = [0, 0, 70]
colormap[15] = [0, 60, 100]
colormap[16] = [0, 80, 100]
colormap[17] = [0, 0, 230]
colormap[18] = [119, 11, 32]

In [15]:
def parent(path):
	path = pathlib.Path(path)
	return str(path.parent)

def exist(path):
	return os.path.exists(str(path))
def mkdir(path):
	pathlib.Path(path).mkdir(parents=True, exist_ok=True)

In [16]:
def convert_seg_gray_to_color(input, n_classes, output_path=None, colors=class_colors):
	"""
	Convert the segmented image on gray to color.
	:param input: it is available to get two type(ndarray, string), string type is a file path.
	:param n_classes: number of the classes.
	:param output_path: output path. if it is None, this function return result array(ndarray)
	:param colors: refer to 'class_colors' format. Default: random assigned color.
	:return: if out_path is None, return result array(ndarray)
	"""
	if isinstance(input, six.string_types):
		seg = cv2.imread(input, flags=cv2.IMREAD_GRAYSCALE)
	elif type(input) is np.ndarray:
		assert len(input.shape) == 2, "Input should be h,w "
		seg = input

	height = seg.shape[0]
	width = seg.shape[1]

	seg_img = np.zeros((height, width, 3))

	for c in range(n_classes):
		seg_arr = seg[:, :] == c
		seg_img[:, :, 0] += ((seg_arr) * colors[c][0]).astype('uint8')
		seg_img[:, :, 1] += ((seg_arr) * colors[c][1]).astype('uint8')
		seg_img[:, :, 2] += ((seg_arr) * colors[c][2]).astype('uint8')

	if output_path:
		cv2.imwrite(output_path, seg_img)
	else:
		return seg_img

In [28]:
def predict(model, input_path, output_path, colors=class_colors):
    model.eval()

    img = cv2.imread(input_path, flags=cv2.IMREAD_COLOR)
    ori_height = img.shape[0]
    ori_width = img.shape[1]

    model_width = 1280
    model_height = 720

    if model_width != ori_width or model_height != ori_height:
        img = cv2.resize(img, (model_width, model_height), interpolation=cv2.INTER_NEAREST)


    data = img.transpose((2, 0, 1))
    data = data[None, :, :, :]
    data = torch.from_numpy(data).float()

    if next(model.parameters()).is_cuda:
        if not torch.cuda.is_available():
            raise ValueError("A model was trained via .cuda(), but this system can not support cuda.")
        data = data.cuda()

    score = model(data)

    lbl_pred = score.data.max(1)[1].cpu().numpy()[:, :, :]
    lbl_pred_ = lbl_pred.transpose((1, 2, 0))
    n_classes = np.max(lbl_pred)
    lbl_pred = lbl_pred_.reshape(model_height, model_width)

    seg_img = convert_seg_gray_to_color(lbl_pred, n_classes, colors=colormap)

    if model_width != ori_width or model_height != ori_height:
        seg_img = cv2.resize(seg_img, (ori_width, ori_height), interpolation=cv2.INTER_NEAREST)

    if not exist(parent(output_path)):
        mkdir(parent(output_path))

    cv2.imwrite(output_path, seg_img)

    return lbl_pred

In [ ]:
for i in range(0,100):
    score = predict(classifier_model, 
        f"/content/gdrive/MyDrive/AI_PROJECT(BLG_527E)/dataset/target/images/{i}.jpg", 
        f'/content/labels/{i}.jpg')